In [ ]:
%pip install datasets torch

In [ ]:
from datasets import load_dataset
d_train = load_dataset("tiny_shakespeare", split="train")
d_val = load_dataset("tiny_shakespeare", split="validation")

In [ ]:
train_data = d_train[0]['text']
val_data = d_val[0]['text']

In [ ]:
chars = sorted(list(set(train_data)))
vocab_size = len(chars)
print(''.join(chars))
print(f"Vocab size: {vocab_size}")

In [ ]:
stoi = { ch: i for i, ch in enumerate(chars)}
itos = { i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
print(encode("Hi there!"))
print(decode(encode("Hi there!")))

In [ ]:
import torch
train_data = torch.tensor(encode(train_data), dtype=torch.long)
print(f"d.shape: {train_data.shape}, d.type: {train_data.dtype}")

In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(f"inputs:")
print(f"{xb.shape}")
print(f"{xb}")
print(f"targets:")
print(f"{yb.shape}")
print(f"{yb}")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is {context.tolist()} the target is {target}")

In [ ]:
print(xb)

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

dev = torch.device("mps")

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)    # B T C

        if targets is None:
            loss = None
        else:
            # Reshape logits
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

In [ ]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

In [ ]:
torch.manual_seed(1337)
B,T,C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

In [ ]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev, 0)